In [1]:
import sys
!"{sys.executable}" -m pip install customtkinter openai langchain langchain-openai


   ---------------------------------------- 0/2 [darkdetect]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   -------------------- ------------------- 1/2 [customtkinter]
   ---------------------------------------- 2/2 [customtkinter]




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import sys
!"{sys.executable}" -m pip install tkcalendar


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached babel-2.17.0-py3-none-any.whl.metadata (2.0 kB)
Using cached babel-2.17.0-py3-none-any.whl (10.2 MB)

   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------- 0/2 [babel]
   ---------------------------------------

In [ ]:
import customtkinter as ctk
from tkinter import messagebox
from tkcalendar import DateEntry
from PIL import Image
from datetime import datetime
from langchain_openai import ChatOpenAI
import os

# Initialize LangChain LLM
llm = ChatOpenAI(model="gpt-4o-mini")  # requires OPENAI_API_KEY set in environment

# =============== Setup ===============
ctk.set_appearance_mode("System")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.title("Personalized Activity Guide")
app.geometry("650x700")

# =============== Title + Icon ===============
header_frame = ctk.CTkFrame(app)
header_frame.pack(pady=15)

# Convert your calendar.svg → calendar.png before
calendar_img = ctk.CTkImage(
    light_image=Image.open("calendar.png"),
    dark_image=Image.open("calendar.png"),
    size=(50, 50)
)

calendar_label = ctk.CTkLabel(header_frame, image=calendar_img, text="")
calendar_label.pack(side="left", padx=10)

title = ctk.CTkLabel(
    header_frame,
    text="Personalized Activity Guide",
    font=("Arial", 22, "bold")
)
title.pack(side="left")

subtitle = ctk.CTkLabel(
    app,
    text="Generates custom guest guides based on stay details, interests, and transportation preferences.",
    wraplength=500,
    justify="center"
)
subtitle.pack(pady=5)

# =============== Guest Info ===============
guest_frame = ctk.CTkFrame(app)
guest_frame.pack(pady=10, padx=20, fill="x")

fname = ctk.CTkEntry(guest_frame, placeholder_text="First Name", width=150)
fname.grid(row=0, column=0, padx=5, pady=5)

lname = ctk.CTkEntry(guest_frame, placeholder_text="Last Name", width=150)
lname.grid(row=0, column=1, padx=5, pady=5)

guests = ctk.CTkEntry(guest_frame, placeholder_text="# Guests", width=80)
guests.grid(row=0, column=2, padx=5, pady=5)

destination = ctk.CTkEntry(guest_frame, placeholder_text="Destination", width=200)
destination.grid(row=0, column=3, padx=5, pady=5)

# =============== Date Selection ===============
date_frame = ctk.CTkFrame(app)
date_frame.pack(pady=10, padx=20, fill="x")

checkin_label = ctk.CTkLabel(date_frame, text="Check In:")
checkin_label.grid(row=0, column=0, padx=5)
checkin = DateEntry(date_frame, width=12, background='darkblue', foreground='white', borderwidth=2)
checkin.grid(row=0, column=1, padx=5, pady=5)

checkout_label = ctk.CTkLabel(date_frame, text="Check Out:")
checkout_label.grid(row=0, column=2, padx=5)
checkout = DateEntry(date_frame, width=12, background='darkblue', foreground='white', borderwidth=2)
checkout.grid(row=0, column=3, padx=5, pady=5)

# =============== Preferences ===============
prefs_frame = ctk.CTkFrame(app)
prefs_frame.pack(pady=10, padx=20, fill="x")

activity_label = ctk.CTkLabel(prefs_frame, text="Activity Preferences", font=("Arial", 14, "bold"))
activity_label.grid(row=0, column=0, sticky="w", padx=20)

trans_label = ctk.CTkLabel(prefs_frame, text="Transportation Preferences", font=("Arial", 14, "bold"))
trans_label.grid(row=0, column=1, sticky="w", padx=20)

# Activity Checkboxes
activities = ["Relaxation", "Adventure", "Family-Friendly", "Cultural", "Food & Drink"]
activity_vars = {}
for i, act in enumerate(activities):
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(prefs_frame, text=act, variable=var)
    chk.grid(row=i+1, column=0, sticky="w", padx=20)
    activity_vars[act] = var

# Transport Checkboxes
transports = ["Car Rental", "Ride Share", "Metro / Bus", "Bike", "Walk"]
transport_vars = {}
for i, trans in enumerate(transports):
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(prefs_frame, text=trans, variable=var)
    chk.grid(row=i+1, column=1, sticky="w", padx=20)
    transport_vars[trans] = var

# =============== Generate Guide ===============
def generate_guide():
    try:
        # Get form inputs
        first_name = fname.get().strip()
        last_name = lname.get().strip()
        guest_count = guests.get()
        checkin_date = checkin.get_date()
        checkout_date = checkout.get_date()
        destination_value = destination.get().strip()

        # Validate date range
        if checkout_date <= checkin_date:
            messagebox.showerror("Invalid Dates", "❌ Check-out must be after check-in.")
            return

        # Compute days
        days = (checkout_date - checkin_date).days

        # Collect selected activities & transport
        selected_activities = [a for a, v in activity_vars.items() if v.get()]
        selected_transports = [t for t, v in transport_vars.items() if v.get()]

        # Build AI prompt
        prompt = f"""
        Create a travel activity guide for {destination_value} lasting {days} days.
        Include daily recommendations for sightseeing, food, and culture.
        Traveler: {first_name} {last_name}
        Number of Guests: {guest_count}
        Activities Chosen: {", ".join(selected_activities) or "None"}
        Transport Preferences: {", ".join(selected_transports) or "None"}
        """

        # Call LLM
        result = llm.invoke(prompt).content.strip()

        # Save to Markdown file
        filename = f"travel_guide_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(result)

        # Show result in a new window
        result_window = ctk.CTkToplevel(app)
        result_window.title("Generated Guide")
        result_window.geometry("600x500")

        result_box = ctk.CTkTextbox(result_window, width=550, height=400, wrap="word")
        result_box.pack(pady=15, padx=15, fill="both", expand=True)
        result_box.insert("1.0", result)
        result_box.configure(state="disabled")  # make read-only

        # Success message
        messagebox.showinfo("Saved", f"✅ Travel guide saved as {filename}")
        print(f"Guide saved as {filename}")
        app.destroy()

    except Exception as e:
        messagebox.showerror("Error", f"⚠️ Something went wrong:\n{str(e)}")
        app.destroy()

btn = ctk.CTkButton(app, text="⚡ Generate Guide", command=generate_guide)
btn.pack(pady=20)

app.mainloop()


### Monochrome design 

In [10]:
import customtkinter as ctk
from tkinter import messagebox
from tkcalendar import DateEntry
from PIL import Image
from datetime import datetime
from langchain_openai import ChatOpenAI
import os

# Initialize LangChain LLM
llm = ChatOpenAI(model="gpt-4o-mini")  # requires OPENAI_API_KEY set in environment

# =============== Setup ===============
ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

# Custom monochrome color scheme
# COLORS = {
#     'bg_primary': '#1a1a1a',      # Very dark gray
#     'bg_secondary': '#2d2d2d',    # Dark gray
#     'bg_tertiary': '#3d3d3d',     # Medium gray
#     'text_primary': '#ffffff',     # White
#     'text_secondary': '#b0b0b0',   # Light gray
#     'accent': '#808080',           # Medium gray accent
#     'border': '#404040'            # Border gray
# }

COLORS = {
    'bg_primary': '#f7f7f7',      # Very light gray
    'bg_secondary': '#ededed',    # Light gray
    'bg_tertiary': '#d6d6d6',     # Medium-light gray
    'text_primary': '#222222',    # Near black
    'text_secondary': '#555555',  # Medium gray
    'accent': '#b0b0b0',          # Accent gray
    'border': '#cccccc'           # Border gray
}

app = ctk.CTk()
app.title("Personalized Activity Guide")
app.geometry("700x800")
app.configure(fg_color=COLORS['bg_primary'])

# =============== Main Container ===============
main_container = ctk.CTkScrollableFrame(
    app, 
    fg_color=COLORS['bg_primary'],
    scrollbar_fg_color=COLORS['bg_secondary']
)
main_container.pack(fill="both", expand=True, padx=20, pady=20)

# =============== Header Section ===============
header_frame = ctk.CTkFrame(
    main_container,
    fg_color=COLORS['bg_secondary'],
    corner_radius=15,
    border_width=1,
    border_color=COLORS['border']
)
header_frame.pack(fill="x", pady=(0, 20))

# Title with enhanced typography
title = ctk.CTkLabel(
    header_frame,
    text="✈ PERSONALIZED ACTIVITY GUIDE",
    font=("SF Pro Display", 28, "bold"),
    text_color=COLORS['text_primary']
)
title.pack(pady=(20, 5))

subtitle = ctk.CTkLabel(
    header_frame,
    text="Create custom travel itineraries tailored to your preferences",
    font=("SF Pro Display", 14),
    text_color=COLORS['text_secondary'],
    wraplength=600
)
subtitle.pack(pady=(0, 20))

# =============== Guest Information Section ===============
guest_section = ctk.CTkFrame(
    main_container,
    fg_color=COLORS['bg_secondary'],
    corner_radius=15,
    border_width=1,
    border_color=COLORS['border']
)
guest_section.pack(fill="x", pady=(0, 15))

guest_title = ctk.CTkLabel(
    guest_section,
    text="GUEST INFORMATION",
    font=("SF Pro Display", 16, "bold"),
    text_color=COLORS['text_primary']
)
guest_title.pack(pady=(15, 10))

# Guest info grid
guest_grid = ctk.CTkFrame(guest_section, fg_color="transparent")
guest_grid.pack(padx=20, pady=(0, 15))

# Configure grid weights for responsive layout
guest_grid.grid_columnconfigure((0, 1, 2, 3), weight=1)

fname = ctk.CTkEntry(
    guest_grid, 
    placeholder_text="First Name",
    height=40,
    font=("SF Pro Display", 12),
    fg_color=COLORS['bg_tertiary'],
    border_color=COLORS['border'],
    text_color=COLORS['text_primary']
)
fname.grid(row=0, column=0, padx=(0, 8), pady=5, sticky="ew")

lname = ctk.CTkEntry(
    guest_grid,
    placeholder_text="Last Name",
    height=40,
    font=("SF Pro Display", 12),
    fg_color=COLORS['bg_tertiary'],
    border_color=COLORS['border'],
    text_color=COLORS['text_primary']
)
lname.grid(row=0, column=1, padx=4, pady=5, sticky="ew")

guests = ctk.CTkEntry(
    guest_grid,
    placeholder_text="Guests",
    height=40,
    font=("SF Pro Display", 12),
    fg_color=COLORS['bg_tertiary'],
    border_color=COLORS['border'],
    text_color=COLORS['text_primary']
)
guests.grid(row=0, column=2, padx=4, pady=5, sticky="ew")

destination = ctk.CTkEntry(
    guest_grid,
    placeholder_text="Destination City",
    height=40,
    font=("SF Pro Display", 12),
    fg_color=COLORS['bg_tertiary'],
    border_color=COLORS['border'],
    text_color=COLORS['text_primary']
)
destination.grid(row=0, column=3, padx=(8, 0), pady=5, sticky="ew")

# =============== Date Selection Section ===============
date_section = ctk.CTkFrame(
    main_container,
    fg_color=COLORS['bg_secondary'],
    corner_radius=15,
    border_width=1,
    border_color=COLORS['border']
)
date_section.pack(fill="x", pady=(0, 15))

date_title = ctk.CTkLabel(
    date_section,
    text="TRAVEL DATES",
    font=("SF Pro Display", 16, "bold"),
    text_color=COLORS['text_primary']
)
date_title.pack(pady=(15, 10))

date_grid = ctk.CTkFrame(date_section, fg_color="transparent")
date_grid.pack(padx=40, pady=(0, 15))

# Configure grid for centered layout
date_grid.grid_columnconfigure((0, 1, 2, 3), weight=1)

checkin_label = ctk.CTkLabel(
    date_grid,
    text="Check In",
    font=("SF Pro Display", 12, "bold"),
    text_color=COLORS['text_secondary']
)
checkin_label.grid(row=0, column=0, padx=10, pady=(0, 5))

checkin = DateEntry(
    date_grid,
    width=15,
    background='#2d2d2d',
    foreground='white',
    borderwidth=1,
    font=("SF Pro Display", 11)
)
checkin.grid(row=1, column=0, padx=10, pady=5)

checkout_label = ctk.CTkLabel(
    date_grid,
    text="Check Out",
    font=("SF Pro Display", 12, "bold"),
    text_color=COLORS['text_secondary']
)
checkout_label.grid(row=0, column=2, padx=10, pady=(0, 5))

checkout = DateEntry(
    date_grid,
    width=15,
    background='#2d2d2d',
    foreground='white',
    borderwidth=1,
    font=("SF Pro Display", 11)
)
checkout.grid(row=1, column=2, padx=10, pady=5)

# =============== Preferences Section ===============
prefs_section = ctk.CTkFrame(
    main_container,
    fg_color=COLORS['bg_secondary'],
    corner_radius=15,
    border_width=1,
    border_color=COLORS['border']
)
prefs_section.pack(fill="x", pady=(0, 15))

prefs_title = ctk.CTkLabel(
    prefs_section,
    text="PREFERENCES",
    font=("SF Pro Display", 16, "bold"),
    text_color=COLORS['text_primary']
)
prefs_title.pack(pady=(15, 10))

# Preferences grid
prefs_grid = ctk.CTkFrame(prefs_section, fg_color="transparent")
prefs_grid.pack(padx=20, pady=(0, 20), fill="x")

# Configure grid layout
prefs_grid.grid_columnconfigure((0, 1), weight=1)

# Activity Preferences Column
activity_frame = ctk.CTkFrame(
    prefs_grid,
    fg_color=COLORS['bg_tertiary'],
    corner_radius=10
)
activity_frame.grid(row=0, column=0, padx=(0, 10), pady=5, sticky="ew")

activity_label = ctk.CTkLabel(
    activity_frame,
    text="Activity Interests",
    font=("SF Pro Display", 14, "bold"),
    text_color=COLORS['text_primary']
)
activity_label.pack(pady=(15, 10))

# Activity Checkboxes with modern styling
activities = ["🏖 Relaxation", "🏔 Adventure", "👨‍👩‍👧‍👦 Family-Friendly", "🏛 Cultural", "🍽 Food & Drink"]
activity_vars = {}
for act in activities:
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(
        activity_frame,
        text=act,
        variable=var,
        font=("SF Pro Display", 12),
        text_color=COLORS['text_primary'],
        fg_color=COLORS['accent'],
        hover_color=COLORS['text_secondary'],
        checkmark_color=COLORS['bg_primary']
    )
    chk.pack(pady=3, padx=15, anchor="w")
    activity_vars[act] = var

# Transportation Preferences Column
transport_frame = ctk.CTkFrame(
    prefs_grid,
    fg_color=COLORS['bg_tertiary'],
    corner_radius=10
)
transport_frame.grid(row=0, column=1, padx=(10, 0), pady=5, sticky="ew")

transport_label = ctk.CTkLabel(
    transport_frame,
    text="Transportation",
    font=("SF Pro Display", 14, "bold"),
    text_color=COLORS['text_primary']
)
transport_label.pack(pady=(15, 10))

# Transport Checkboxes with icons
transports = ["🚗 Car Rental", "🚕 Ride Share", "🚇 Metro / Bus", "🚲 Bike", "🚶 Walk"]
transport_vars = {}
for trans in transports:
    var = ctk.BooleanVar()
    chk = ctk.CTkCheckBox(
        transport_frame,
        text=trans,
        variable=var,
        font=("SF Pro Display", 12),
        text_color=COLORS['text_primary'],
        fg_color=COLORS['accent'],
        hover_color=COLORS['text_secondary'],
        checkmark_color=COLORS['bg_primary']
    )
    chk.pack(pady=3, padx=15, anchor="w")
    transport_vars[trans] = var

# =============== Generate Button ===============
def generate_guide():
    try:
        # Get form inputs
        first_name = fname.get().strip()
        last_name = lname.get().strip()
        guest_count = guests.get()
        checkin_date = checkin.get_date()
        checkout_date = checkout.get_date()
        destination_value = destination.get().strip()

        # Validation
        if not first_name or not destination_value:
            messagebox.showerror("Missing Information", "❌ Please fill in at least First Name and Destination.")
            return

        if checkout_date <= checkin_date:
            messagebox.showerror("Invalid Dates", "❌ Check-out must be after check-in.")
            return

        # Compute days
        days = (checkout_date - checkin_date).days

        # Collect selected preferences
        selected_activities = [a for a, v in activity_vars.items() if v.get()]
        selected_transports = [t for t, v in transport_vars.items() if v.get()]

        # Build AI prompt
        prompt = f"""
        Create a comprehensive travel activity guide for {destination_value} lasting {days} days.
        Include daily recommendations for sightseeing, dining, and cultural experiences.
        
        Traveler Details:
        - Name: {first_name} {last_name}
        - Party Size: {guest_count} guests
        - Dates: {checkin_date} to {checkout_date}
        
        Preferences:
        - Activities: {", ".join(selected_activities) if selected_activities else "Open to all experiences"}
        - Transportation: {", ".join(selected_transports) if selected_transports else "Flexible transportation options"}
        
        Please format the guide with clear day-by-day breakdowns and practical tips.
        """

        # Show loading state
        btn.configure(text="🔄 Generating Guide...", state="disabled")
        app.update()

        # Call LLM
        result = llm.invoke(prompt).content.strip()

        # Save to Markdown file
        filename = f"travel_guide_{first_name}_{last_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"# Travel Guide for {first_name} {last_name}\n")
            f.write(f"**Destination:** {destination_value}\n")
            f.write(f"**Dates:** {checkin_date} to {checkout_date}\n")
            f.write(f"**Duration:** {days} days\n\n")
            f.write("---\n\n")
            f.write(result)

        # Show result in enhanced window
        show_result_window(result, filename, first_name, destination_value, days)
        messagebox.showinfo("Saved", f"✅ Travel guide saved as {filename}")
        print(f"Guide saved as {filename}")

        # Reset button
        btn.configure(text="✨ Generate Guide", state="normal")

    except Exception as e:
        btn.configure(text="✨ Generate Guide", state="normal")
        messagebox.showerror("Error", f"⚠️ Something went wrong:\n{str(e)}")

def show_result_window(content, filename, name, destination, days):
    result_window = ctk.CTkToplevel(app)
    result_window.title(f"Travel Guide - {name}")
    result_window.geometry("800x600")
    result_window.configure(fg_color=COLORS['bg_primary'])
    
    # Header in result window
    result_header = ctk.CTkFrame(
        result_window,
        fg_color=COLORS['bg_secondary'],
        corner_radius=10
    )
    result_header.pack(fill="x", padx=20, pady=(20, 10))
    
    result_title = ctk.CTkLabel(
        result_header,
        text=f"🗺 {destination} • {days} Days",
        font=("SF Pro Display", 20, "bold"),
        text_color=COLORS['text_primary']
    )
    result_title.pack(pady=15)
    
    # Content area
    content_frame = ctk.CTkFrame(
        result_window,
        fg_color=COLORS['bg_secondary'],
        corner_radius=10
    )
    content_frame.pack(fill="both", expand=True, padx=20, pady=(0, 20))
    
    result_box = ctk.CTkTextbox(
        content_frame,
        font=("SF Pro Display", 12),
        fg_color=COLORS['bg_tertiary'],
        text_color=COLORS['text_primary'],
        border_color=COLORS['border'],
        wrap="word"
    )
    result_box.pack(fill="both", expand=True, padx=15, pady=15)
    result_box.insert("1.0", content)
    result_box.configure(state="disabled")
    
    # Action buttons
    button_frame = ctk.CTkFrame(result_window, fg_color="transparent")
    button_frame.pack(pady=(0, 15))
    
    save_btn = ctk.CTkButton(
        button_frame,
        text=f"📄 Saved as {filename}",
        font=("SF Pro Display", 12, "bold"),
        fg_color=COLORS['accent'],
        hover_color=COLORS['text_secondary'],
        text_color=COLORS['text_primary'],
        height=35,
        corner_radius=8,
        state="enabled"
    )
    save_btn.pack(side="left", padx=5)
    
    close_btn = ctk.CTkButton(
        button_frame,
        text="✕ Close",
        font=("SF Pro Display", 12, "bold"),
        fg_color=COLORS['bg_tertiary'],
        hover_color=COLORS['accent'],
        text_color=COLORS['text_primary'],
        height=35,
        corner_radius=8,
        command=lambda: app.destroy())
    
    close_btn.pack(side="left", padx=5)

# Generate button with enhanced styling
button_frame = ctk.CTkFrame(main_container, fg_color="transparent")
button_frame.pack(pady=20)

btn = ctk.CTkButton(
    button_frame,
    text="✨ Generate Guide",
    command=generate_guide,
    font=("SF Pro Display", 16, "bold"),
    height=50,
    width=200,
    fg_color=COLORS['text_primary'],
    hover_color=COLORS['text_secondary'],
    text_color=COLORS['bg_primary'],
    corner_radius=8
)
btn.pack()

# Footer
footer = ctk.CTkLabel(
    main_container,
    text="Powered by AI • Custom travel recommendations",
    font=("SF Pro Display", 10),
    text_color=COLORS['text_secondary']
)
footer.pack(pady=(10, 0))

# =============== Input Validation ===============
def validate_numeric_input(event):
    """Ensure guests field only accepts numbers"""
    current = guests.get()
    if current and not current.isdigit():
        guests.delete(len(current)-1)

guests.bind('<KeyRelease>', validate_numeric_input)

# =============== Enhanced DateEntry Styling ===============
def style_date_entries():
    """Apply monochrome styling to date entries"""
    try:
        checkin.configure(
            background=COLORS['bg_tertiary'],
            foreground=COLORS['text_primary'],
            fieldbackground=COLORS['bg_tertiary'],
            bordercolor=COLORS['border'],
            arrowcolor=COLORS['text_secondary']
        )
        checkout.configure(
            background=COLORS['bg_tertiary'],
            foreground=COLORS['text_primary'],
            fieldbackground=COLORS['bg_tertiary'],
            bordercolor=COLORS['border'],
            arrowcolor=COLORS['text_secondary']
        )
    except:
        pass  # Some styling options might not be available

# Apply styling after widget creation
app.after(100, style_date_entries)

# =============== Keyboard Shortcuts ===============
def on_enter_key(event):
    """Generate guide when Enter is pressed"""
    generate_guide()

app.bind('<Return>', on_enter_key)

# =============== Window Settings ===============
app.resizable(True, True)
app.minsize(600, 700)

# Center window on screen
app.update_idletasks()
x = (app.winfo_screenwidth() // 2) - (700 // 2)
y = (app.winfo_screenheight() // 2) - (800 // 2)
app.geometry(f"700x800+{x}+{y}")

if __name__ == "__main__":
    app.mainloop()

Guide saved as travel_guide_Jyde_Eli_20250902_092219.md
